In [131]:
import librosa.core

def parse_dadb(filename):
    data = []
    labels = []
    frames = []
    
    with open(filename,'r') as file:
        line = file.readline()
        k=0
        while line:
            line_split = line.split(',')
            begin_time,end_time = float(line_split[0]),float(line_split[1])
            label = line_split[-9]
            words = [x.split('+')[-1] for x in line_split[4].split('|')]


            data.append(words)
            labels.append(label)
            frames.append(librosa.core.time_to_frames((begin_time,end_time)))

            line = file.readline()
    return data,labels,frames

In [132]:
def mapping_labels(label):
    if '|' in label:
        for i in range(len(label)):
            if label[i]=='|':
                return label[i+1]
    if label=='' or label[0].isdigit():
        return 'z'
    if label[0] in ['s','q','b','%','f']:
        return label[0]
    if label[0] == 'h':
        return 'f'
    if label[0] == 'x':
        return '%'
    return label
set([mapping_labels(label) for label in labels])

{'%', 'b', 'f', 'q', 's', 'z'}

In [133]:
training_set = []
dev_set = []
test_set = []
with open('train_test_split.txt','r') as file:
    line = file.readline()
    train = True
    line = file.readline()
    while line[:3]!='Dev':
        if len(line)>1:
            training_set.append(line[:-1])
        line = file.readline()
    line = file.readline()
    while line[:3]!='Tes':
        if len(line)>1:
            dev_set.append(line[:-1])
        line = file.readline()
    line = file.readline()
    while line[:3]!='Not':
        if len(line)>1:
            test_set.append(line[:-1])
        line = file.readline()

In [134]:
s = {'train':training_set,'dev':dev_set,'test':test_set}
folders = ['train','dev','test']

In [137]:
def build_vocab(folders,s):
    frames_dict = {}
    data_dict = {}
    labels_dict = {}
    vocabulary = {}
    dict_index = 1
    for state in folders:
        for name in s[state]:
            data,labels,frames = parse_dadb('data\\dadb\\' + name + '.dadb')           
            sent_len = max(sent_len,max_sent_len)
            frames_len = max(frames_len,max_frames_len)
            
            for x in frames:
                lengths.append(x[1]-x[0]) 
            #update dictionary
            for sentence in data:
                for word in sentence:
                    try:
                        vocabulary[word]
                    except:
                        vocabulary[word] = dict_index
                        dict_index+=1
                        
            indexed_data = [[vocabulary[word] for word in sentence] for sentence in data]
            indexed_data = [[],[]] + indexed_data
            data_dict[name] = [[indexed_data[k],indexed_data[k+1],indexed_data[k+2]] for k in range(len(data))]
            labels_dict[name] = [mapping_labels(label) for label in labels]
            frames_dict[name] = frames
            
    return vocabulary,data_dict,labels_dict,frames_dict,sent_len,frames_len

In [138]:
vocabulary,data_dict,labels_dict,frames_dict,sent_len,frames_len = build_vocab(folders,s)

In [139]:
sent_len

78

In [240]:
name = 'Bed004'
audio = torch.load('data\\' + 'train'+'\\' + name + '.pt')
data = data_dict[name]
frames = frames_dict[name]
labels = labels_dict[name]
sent_len = 50
audio_len = 500
labels_encoding = {'%':0, 'b':1, 'f':2, 'q':3, 's':4}
audio_tensors = []
text_tensors = []
labels_tensor = []
for i in range(len(data)):
    begin_f = frames[i][0]
    end_f = frames[i][1]
    if end_f - begin_f < 501 and labels[i]!='z' and len(data[i]) < 51:
        audio_tensors.append(padding(audio[:,begin_f:end_f],500))
        text_tensors.append(text_to_torch(data[i],sent_len))
        labels_tensor.append(labels_encoding[labels[i]])
aud = torch.stack(audio_tensors)
text = torch.stack(text_tensors)
lab = torch.tensor(labels_tensor)

In [243]:
text.shape

torch.Size([817, 3, 78])

In [237]:
final_labels[3]

0

In [224]:
def padding_audio(tensor,length):
    i,j = tensor.shape
    objective = torch.zeros(i,length)
    objective[:,:j] = tensor
    return objective

def text_to_torch(text_list,sent_len):
    sentences = [torch.zeros(sent_len) for i in range(3)]
    for i in range(3):
        text = text_list[i]
        sentences[i][:len(text)] = torch.tensor(text)
    return torch.stack(sentences)

In [210]:
text_totorch(final_text[10],78)

tensor([[2.1000e+01, 5.1000e+01, 1.3350e+03, 2.0000e+00, 3.0000e+00, 6.0800e+02,
         1.6880e+03, 8.9000e+01, 3.7600e+02, 8.9000e+01, 6.5000e+01, 5.3000e+01,
         5.2000e+01, 2.3300e+02, 2.6700e+02, 4.4000e+01, 7.0000e+01, 1.2900e+03,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e

In [208]:
X,Y = final_audio[1].shape
Y

2

In [211]:
len(vocabulary)

14599

In [216]:
vocabulary['mmm']

259

In [252]:
a,b,c = [None for i in range(3)]

In [254]:
dataset_train,train_loader,dataset_dev,dev_loader,dataset_test,test_loader = [None for i in range(6)]
variables = {'train':(dataset_train,train_loader),'dev':(dataset_dev,dev_loader),'test':(dataset_test,test_loader)}

In [257]:
variables['train'][0] = 2

TypeError: 'tuple' object does not support item assignment

In [259]:
import baseline.batcher as batcher

ModuleNotFoundError: No module named 'lexical'